In [ ]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [ ]:
load_dotenv()

In [ ]:
conn_string = (
    f"host={os.getenv('hostname')} "
    f"dbname={os.getenv('database')} "
    f"user={os.getenv('username')} "
    f"password={os.getenv('password')} "
    f"port={os.getenv('port')}"
)

In [ ]:
conn = psycopg2.connect(conn_string)

In [ ]:
cursor = conn.cursor()

In [ ]:
schema_name = "public,dadeh"

In [ ]:
cursor.execute(f"SET search_path TO {schema_name}")

In [ ]:
df = pd.read_csv("contrat_inseres.csv", dtype=str)

In [ ]:
employeurs = list(df["id_employeur"].unique())

In [ ]:
dfe = pd.DataFrame([])

In [ ]:
i = 0
cpt = 0
for em in employeurs:
    if cpt == 0:
        str_em = ""
    cpt += 1
    str_em += "id = " + em + " OR "
    if cpt == 100:
        # send
        str_em = str_em[:-3]
        query = (
            "select id, et_code_apet, et_ad_code_postal, et_categorie_juridique, "
            "et_effectif_fin_periode, en_code_apen, en_ad_code_postal, en_effectif_moyen "
            "from ddadtemployeur where " + str_em
        )
        dfinter = pd.read_sql_query(query, conn)
        dfe = pd.concat([dfe, dfinter])
        cpt = 0
        i = i + 1
        print(i * 100)

In [ ]:
dfe

In [ ]:
dfe = dfe.rename(columns={"id": "id_employeur"})

In [ ]:
dfe.to_csv("employeurs_inseres.csv", index=False)

In [ ]:
dfe = pd.read_csv("employeurs_inseres.csv", dtype=str)

In [ ]:
df = pd.merge(df, dfe, on="id_employeur", how="left")

In [ ]:
df.to_csv("contrat_inseres_employeurs.csv", index=False)